In [2]:
from pynq import Overlay

from pynq import allocate
import numpy as np

In [3]:
overlay = Overlay("design_1_wrapper.bit")

In [4]:
overlay?

In [4]:
dma = overlay.axi_dma_0
mov_avg = overlay.moving_average_2_0

In [5]:
mov_avg.register_map

RegisterMap {
  win = Register(win=write-only),
  dim = Register(dim=write-only)
}

In [6]:
dim = 1000
window = 15

In [22]:
mov_avg.register_map.win = window
mov_avg.register_map.dim = dim

In [23]:
data_in = np.random.randint(-1000, 1000, dim, np.int32)
#data_in = np.empty(dim, np.int32)
#for i in range(dim):
#    data_in[i]=i

In [24]:
print("Input data sample:",data_in[0:10])

Input data sample: [ -89  385  486 -663  481  108 -327  740  -18 -501]


In [25]:
# moving average software function
def moving_average(data, window):
    data_out_size = data.shape[0] - window + 1
    data_out = np.empty(data_out_size, np.float32)
    for i in range(data_out_size):
        data_out[i] = np.sum(data[i:i + window])/window
    return data_out 

In [26]:
data_out = moving_average(data_in, window)

In [27]:
print("Software computed output sample: ",data_out[0:10])

Software computed output sample:  [120.2      110.933334 138.2       52.866665 149.06667  169.46666
 173.73334  166.2       89.26667   89.8     ]


In [28]:
# create hardware buffers
buf_in = allocate(dim, np.int32)
buf_out = allocate(dim - window + 1, np.float32)

In [29]:
buf_in[:] = data_in[:]

In [30]:
buf_in.flush()

In [31]:
# send and receive data
dma.recvchannel.transfer(buf_out)
dma.sendchannel.transfer(buf_in)
# wait for data
dma.sendchannel.wait()
dma.recvchannel.wait()

In [32]:
print("Hardware computed output sample: ",buf_out[0:10])

Hardware computed output sample:  [120.2      110.933334 138.2       52.866665 149.06667  169.46666
 173.73334  166.2       89.26667   89.8     ]


In [33]:
buf_out.invalidate()